In [45]:
import xarray as xr 
import numpy as np

# Maths Servers

In [46]:
# import sys
# sys.path.append('/home/links/ct715/eddy_feedback')
# import functions.eddy_feedback as ef 

# ds = xr.open_mfdataset('/home/links/ct715/eddy_feedback/daily_datasets/jra55_djf_ep.nc',
#                             parallel=True, chunks={'time': 31})
# ds

# JASMIN Server

In [47]:
import sys
sys.path.append('/home/users/cturrell/documents/eddy_feedback')
import functions.eddy_feedback as ef 

ds = xr.open_mfdataset('/home/users/cturrell/documents/eddy_feedback/daily_datasets/jra55_djf_ubar_div1.nc',
                            parallel=True, chunks={'time': 31})

# # TEM SPARC Reanalysis
# ds = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/eddy_feedback/reanalysis/jra55/*.nc') 

ds

<xarray.Dataset>
Dimensions:  (time: 5325, level: 37, lat: 73)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01T09:00:00 ... 2016-12-31T09:00:00
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * level    (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    ubar     (time, level, lat) float32 dask.array<chunksize=(31, 37, 73), meta=np.ndarray>
    div1     (time, level, lat) float64 dask.array<chunksize=(31, 37, 73), meta=np.ndarray>

In [48]:
def calculate_ef_parameter(ds, primitive=True, hemisphere='NH', reanalysis=True,
                           whole_area=True, which_level=None, rename_variables=False):

    """ 
    Input: Xarray dataset

    Output: Xarray dataArray of Eddy Feedback Parameter (EFP)
    """

    ## CONDITIONS
    
    # if required, rename variables
    if rename_variables:
        ds = ef.find_rename_variables(ds) 


    # Reduce dataset to 200-600hPa
    ds = ds.where( ds.level >= 200., drop=True ) 
    ds = ds.where( ds.level <= 600., drop=True ) 

    # choose hemisphere
    if hemisphere == 'SH':
        # set southern hermisphere
        ds = ds.where( ds.lat <= 0, drop=True )
    else:
        # set northern hemisphere
        ds = ds.where( ds.lat >= 0, drop=True )


    #-------------------------------------------------------------------

    ## SET UP TIME
    
    # # remove unwanted variables
    # vars = ['u', 'div1', 'div2']
    # ds = ds[vars]
    
    # set variables and save them
    ubar = ds.ubar
    div1 = ds.div1
    # div2 = ds.div2
    
    if reanalysis: 
        # separate time into annual means
        # and use .load() to force the calculation now
        ubar = ubar.groupby('time.year').mean('time').load()
        div1 = div1.groupby('time.year').mean('time').load()
        # div2 = div2.groupby('time.year').mean('time').load()
    else:
        # separate time into annual means
        ubar = ubar.load()
        div1 = div1.load()
        # div2 = div2.load()

    # calculate Pearson's correlation
    R = xr.corr(div1, ubar, dim='year')

    


    # choose between whole area or individual levels
    if whole_area: 
        # average over 200-600hPa
        R = R.mean(('level'))
    else:
        # take a slice at particular level
        R = R.sel( level = which_level)

    # calculate correlation squared (R^2)
    R = R**2


    return R

In [49]:
R = calculate_ef_parameter(ds, whole_area=False, which_level=500)
R

<xarray.DataArray (lat: 37)>
array([3.98460238e-04, 1.35809419e-02, 4.69312393e-02, 3.04861564e-05,
       5.15407160e-03, 3.81765143e-03, 4.12081167e-05, 1.12872633e-04,
       3.11208404e-02, 1.30909960e-01, 2.40787246e-01, 3.57719350e-01,
       3.90621568e-01, 2.60540085e-01, 1.74863480e-01, 1.99294352e-01,
       2.95425551e-01, 3.01305558e-01, 2.45388351e-01, 2.32539284e-01,
       1.76749351e-01, 2.06060484e-01, 4.21796849e-01, 4.25675851e-01,
       3.13915308e-01, 2.42996076e-01, 1.83629589e-01, 1.38133931e-01,
       5.62253841e-02, 3.57668153e-03, 7.71904541e-02, 1.07751810e-01,
       1.22256071e-01, 7.40646278e-02, 3.41732804e-02, 6.43931461e-02,
       4.95746516e-02])
Coordinates:
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... 10.0 7.5 5.0 2.5 0.0
    level    float64 500.0

In [50]:
import numpy as np
RR = R*np.cos( np.deg2rad(ds.lat) )

RR = RR.sel(lat=slice(72,25))

RR

<xarray.DataArray (lat: 19)>
array([0.01064395, 0.05009707, 0.10176109, 0.16517641, 0.19531078,
       0.13998809, 0.10029757, 0.12132271, 0.18989588, 0.20355908,
       0.17351577, 0.17144594, 0.13539786, 0.16347877, 0.34551575,
       0.35901137, 0.27185863, 0.21554015, 0.16642493])
Coordinates:
  * lat      (lat) float64 70.0 67.5 65.0 62.5 60.0 ... 35.0 32.5 30.0 27.5 25.0
    level    float64 500.0

In [51]:
R_weighted = RR.sum('lat') / (np.cos( np.deg2rad(ds.lat) )).sum('lat')
R_weighted.values 

array(0.07157513)

# SPARC Reanalysis 

In [52]:
ds = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/eddy_feedback/reanalysis/jra55_djf_TEM_ubar_div1.nc')
ds

<xarray.Dataset>
Dimensions:  (time: 177, level: 22, lat: 73)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-14T09:00:00 ... 2016-12-14T09:00:00
  * level    (level) float32 1e+03 925.0 850.0 700.0 600.0 ... 5.0 3.0 2.0 1.0
  * lat      (lat) float32 -90.0 -87.5 -85.0 -82.5 -80.0 ... 82.5 85.0 87.5 90.0
Data variables:
    ubar     (time, level, lat) float32 dask.array<chunksize=(177, 22, 73), meta=np.ndarray>
    div1     (time, level, lat) float32 dask.array<chunksize=(177, 22, 73), meta=np.ndarray>

In [53]:
# correlation_contourf(ds)

In [54]:
RR = calculate_ef_parameter(ds)
RR 

<xarray.DataArray (lat: 37)>
array([1.89410153e-03, 3.77905707e-03, 7.27762498e-03, 1.00144587e-02,
       1.00208034e-02, 1.83465397e-03, 2.17027720e-04, 2.42362933e-03,
       5.12267184e-02, 1.29679948e-01, 2.09810739e-01, 2.86876167e-01,
       3.07895853e-01, 3.92260746e-01, 4.45073713e-01, 3.44805309e-01,
       2.59255152e-01, 2.42707871e-01, 2.59198353e-01, 3.39388377e-01,
       4.02213627e-01, 3.89806989e-01, 3.74076655e-01, 4.14344675e-01,
       4.78718504e-01, 4.62955682e-01, 3.48621158e-01, 2.12845702e-01,
       8.93615681e-02, 1.53608029e-02, 5.63205648e-03, 1.62914480e-02,
       3.35378583e-02, 1.19149363e-02, 3.32400539e-03,            nan,
                  nan])
Coordinates:
  * lat      (lat) float32 0.0 2.5 5.0 7.5 10.0 ... 80.0 82.5 85.0 87.5 90.0

In [55]:
RR = RR*np.cos( np.deg2rad(ds.lat) )

RR = RR.sel(lat=slice(25,72))

RR

<xarray.DataArray (lat: 19)>
array([0.1901531 , 0.25446226, 0.26664563, 0.33082935, 0.36458305,
       0.27355244, 0.19860097, 0.17894301, 0.18328091, 0.22928747,
       0.25853795, 0.23729946, 0.21456156, 0.22262722, 0.23935924,
       0.21376914, 0.14733366, 0.08145252, 0.03056346])
Coordinates:
  * lat      (lat) float32 25.0 27.5 30.0 32.5 35.0 ... 60.0 62.5 65.0 67.5 70.0

In [56]:
R_weighted = RR.sum('lat') / (np.cos( np.deg2rad(ds.lat) )).sum('lat')
R_weighted.values

array(0.089808)